In [128]:
import pandas as pd
import numpy as np

In [180]:
data1 = pd.read_csv("files\\data\\1_food_waste.csv", encoding='cp949')
data1.head()

,base_date,city,emd_cd,emd_nm,em_area_cd,em_cnt,em_g,pay_amt
0,2018-01-01,제주시,50110670,외도동,W6X062,2,15500,464
1,2018-01-01,제주시,50110630,아라동,W6XA97,25,59800,1787
2,2018-01-01,제주시,50110630,아라동,W6XA96,8,25350,758
3,2018-01-01,제주시,50110630,아라동,W6XA95,12,30000,898
4,2018-01-01,제주시,50110630,아라동,W6XA94,14,44050,1317


In [156]:
# 필요없는 열 삭제 (날짜, 행정동코드, 행정동명, 배출량만 남김)
data1_pre = data1.drop(['city', 'em_area_cd', 'pay_amt'], 1)
data1_pre

,base_date,emd_cd,emd_nm,em_cnt,em_g
0,2018-01-01,50110670,외도동,2,15500
1,2018-01-01,50110630,아라동,25,59800
2,2018-01-01,50110630,아라동,8,25350
3,2018-01-01,50110630,아라동,12,30000
4,2018-01-01,50110630,아라동,14,44050
...,...,...,...,...,...
2211440,2021-06-30,50110250,한림읍,12,58250
2211441,2021-06-30,50110250,한림읍,15,111000
2211442,2021-06-30,50110250,한림읍,21,48300
2211443,2021-06-30,50110250,한림읍,5,22050


In [158]:
# 각 열을 하나의 type으로 통일

# 날짜데이터 datetime으로 변환
data1_pre['base_date'] = pd.to_datetime(data1_pre['base_date'], format="%Y-%m-%d")
data1_pre.set_index(data1_pre['base_date'])
# "알수없음" 삭제
idx = data1_pre[data1_pre['emd_cd'] == "알수없음"].index
data1_pre = data1_pre.drop(idx)
# 행정동코드 int형으로 변환
data1_pre['emd_cd'] = pd.to_numeric(data1_pre['emd_cd'])
# 배출건수 int형으로 변환
data1_pre['em_cnt'] = pd.to_numeric(data1_pre['em_cnt'])
# 배출량 int형으로 변환
data1_pre['em_g'] = pd.to_numeric(data1_pre['em_g'])

data1_pre

,base_date,emd_cd,emd_nm,em_cnt,em_g
0,2018-01-01,50110670,외도동,2,15500
1,2018-01-01,50110630,아라동,25,59800
2,2018-01-01,50110630,아라동,8,25350
3,2018-01-01,50110630,아라동,12,30000
4,2018-01-01,50110630,아라동,14,44050
...,...,...,...,...,...
2211440,2021-06-30,50110250,한림읍,12,58250
2211441,2021-06-30,50110250,한림읍,15,111000
2211442,2021-06-30,50110250,한림읍,21,48300
2211443,2021-06-30,50110250,한림읍,5,22050


In [159]:
# 행정동코드 매개변수로 받아서 해당 구역의 배출량을 월별로 정리
def monthly_output(emdcd):
    emd_data = data1_pre[data1_pre['emd_cd'] == emdcd]    # 해당 행정동 데이터만 추출
    if len(emd_data != 0):
        emd_data_monthly = emd_data.resample('M', on='base_date').sum()    # 월별로 합산
        emd_data_monthly['emd_cd'] = emdcd
        return emd_data_monthly
    else:
        return

In [170]:
emd_cd_ary = [50110250, 50110253, 50110256, 50110259, 50110310, 50110320, 50110330, 50110510, 50110520, 50110530, 50110540, 50110550, 50110560, 50110570, 50110580, 50110590, 50110600, 50110610, 50110620, 50110630, 50110640, 50110650, 50110660, 50110670, 50110680, 50110690, 50130250, 50130253, 50130259, 50130310, 50130320, 50130510, 50130520, 50130530, 50130540, 50130550, 50130560, 50130570, 50130580, 50130590, 50130600, 50130610, 50130620]

data1_result = pd.DataFrame(index=range(0,0), columns=['base_date', 'emd_cd', 'em_cnt', 'em_g'])

for i in emd_cd_ary:
    tmp = monthly_output(i)
    data1_result = data1_result.append(tmp)

data1_result['base_date'] = data1_result.index
data1_result

,base_date,emd_cd,em_cnt,em_g
2019-11-30,2019-11-30,50110250,620,2279550
2019-12-31,2019-12-31,50110250,8178,31493650
2020-01-31,2020-01-31,50110250,13042,49632850
2020-02-29,2020-02-29,50110250,12264,46197050
2020-03-31,2020-03-31,50110250,14316,52561750
...,...,...,...,...
2021-02-28,2021-02-28,50130620,4563,15566250
2021-03-31,2021-03-31,50130620,4920,17300600
2021-04-30,2021-04-30,50130620,5256,17646850
2021-05-31,2021-05-31,50130620,6346,19494750


In [176]:
data1_result.to_csv("files\\data_preprocessing\\food_waste_preprocessing.csv", index_label=False)

In [179]:
pd.read_csv("files\\data_preprocessing\\food_waste_preprocessing.csv")

,base_date,emd_cd,em_cnt,em_g
2019-11-30,2019-11-30,50110250,620,2279550
2019-12-31,2019-12-31,50110250,8178,31493650
2020-01-31,2020-01-31,50110250,13042,49632850
2020-02-29,2020-02-29,50110250,12264,46197050
2020-03-31,2020-03-31,50110250,14316,52561750
...,...,...,...,...
2021-02-28,2021-02-28,50130620,4563,15566250
2021-03-31,2021-03-31,50130620,4920,17300600
2021-04-30,2021-04-30,50130620,5256,17646850
2021-05-31,2021-05-31,50130620,6346,19494750
